In [1]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import datetime

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/18 03:30:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.version

'3.3.2'

In [4]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-18 03:30:30--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.160.201.50, 18.160.201.5, 18.160.201.126, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.160.201.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet.1’

yellow_tripdata_202 100%[===================>]  61.36M   200MB/s    in 0.3s    

2025-03-18 03:30:31 (200 MB/s) - ‘yellow_tripdata_2024-10.parquet.1’ saved [64346071/64346071]



In [5]:
!ls -lh

total 123M
-rw-rw-r-- 1 guilhermesemissatto guilhermesemissatto  16K Mar 18 03:26 Untitled.ipynb
-rw-rw-r-- 1 guilhermesemissatto guilhermesemissatto 7.2K Mar 18 03:28 homework5.ipynb
drwxr-xr-x 3 guilhermesemissatto guilhermesemissatto 4.0K Mar 18 02:58 pq
-rw-rw-r-- 1 guilhermesemissatto guilhermesemissatto  13K Feb 22  2024 taxi_zone_lookup.csv
-rw-rw-r-- 1 guilhermesemissatto guilhermesemissatto  62M Dec 18 21:21 yellow_tripdata_2024-10.parquet
-rw-rw-r-- 1 guilhermesemissatto guilhermesemissatto  62M Dec 18 21:21 yellow_tripdata_2024-10.parquet.1


In [6]:
df = (
    spark
    .read
    .parquet('yellow_tripdata_2024-10.parquet')
)

In [7]:
df.count()

3833771

In [10]:
df_repartitioned = (
    df
    .repartition(4)
    .write
    .parquet('pq/df_repartitioned', mode='overwrite')
)

In [11]:
!ls -lh

total 123M
-rw-rw-r-- 1 guilhermesemissatto guilhermesemissatto  16K Mar 18 03:26 Untitled.ipynb
-rw-rw-r-- 1 guilhermesemissatto guilhermesemissatto 7.2K Mar 18 03:28 homework5.ipynb
drwxr-xr-x 3 guilhermesemissatto guilhermesemissatto 4.0K Mar 18 03:31 pq
-rw-rw-r-- 1 guilhermesemissatto guilhermesemissatto  13K Feb 22  2024 taxi_zone_lookup.csv
-rw-rw-r-- 1 guilhermesemissatto guilhermesemissatto  62M Dec 18 21:21 yellow_tripdata_2024-10.parquet
-rw-rw-r-- 1 guilhermesemissatto guilhermesemissatto  62M Dec 18 21:21 yellow_tripdata_2024-10.parquet.1


In [14]:
!cd pq/df_repartitioned

In [15]:
!ls -lh

total 123M
-rw-rw-r-- 1 guilhermesemissatto guilhermesemissatto  16K Mar 18 03:26 Untitled.ipynb
-rw-rw-r-- 1 guilhermesemissatto guilhermesemissatto 7.2K Mar 18 03:28 homework5.ipynb
drwxr-xr-x 3 guilhermesemissatto guilhermesemissatto 4.0K Mar 18 03:31 pq
-rw-rw-r-- 1 guilhermesemissatto guilhermesemissatto  13K Feb 22  2024 taxi_zone_lookup.csv
-rw-rw-r-- 1 guilhermesemissatto guilhermesemissatto  62M Dec 18 21:21 yellow_tripdata_2024-10.parquet
-rw-rw-r-- 1 guilhermesemissatto guilhermesemissatto  62M Dec 18 21:21 yellow_tripdata_2024-10.parquet.1


In [16]:
df.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [17]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [18]:
analisis = (
    df
    .withColumn('tpep_pickup_date', F.to_date('tpep_pickup_datetime'))
    .groupby('tpep_pickup_date')
    .agg(F.count("*").alias('count'))
    .orderBy('tpep_pickup_date')
)

In [19]:
analisis.show()

+----------------+------+
|tpep_pickup_date| count|
+----------------+------+
|      2009-01-01|     1|
|      2024-09-30|    12|
|      2024-10-01|119118|
|      2024-10-02|113906|
|      2024-10-03|108847|
|      2024-10-04|112431|
|      2024-10-05|124443|
|      2024-10-06|103627|
|      2024-10-07|102014|
|      2024-10-08|121402|
|      2024-10-09|129915|
|      2024-10-10|143206|
|      2024-10-11|128821|
|      2024-10-12|130159|
|      2024-10-13|109969|
|      2024-10-14|101513|
|      2024-10-15|128893|
|      2024-10-16|134891|
|      2024-10-17|136330|
|      2024-10-18|133213|
+----------------+------+
only showing top 20 rows



In [20]:
analisis2 = (
    df
    .withColumn('tpep_dropoff_date', F.to_date('tpep_dropoff_datetime'))
    .groupby('tpep_dropoff_date')
    .agg(F.count("*").alias('count'))
    .orderBy('tpep_dropoff_date')
)

In [21]:
analisis2.show()

+-----------------+------+
|tpep_dropoff_date| count|
+-----------------+------+
|       2009-01-01|     1|
|       2024-09-30|     3|
|       2024-10-01|118439|
|       2024-10-02|113803|
|       2024-10-03|108704|
|       2024-10-04|111523|
|       2024-10-05|124131|
|       2024-10-06|105279|
|       2024-10-07|101839|
|       2024-10-08|121304|
|       2024-10-09|129728|
|       2024-10-10|142495|
|       2024-10-11|128717|
|       2024-10-12|130009|
|       2024-10-13|111074|
|       2024-10-14|101517|
|       2024-10-15|128811|
|       2024-10-16|134584|
|       2024-10-17|135907|
|       2024-10-18|132641|
+-----------------+------+
only showing top 20 rows



In [22]:
analisis3 = (
    df
    .withColumn('tpep_dropoff_date', F.to_date('tpep_dropoff_datetime'))
    .where(F.col('tpep_dropoff_date') == datetime.date(2024, 10, 15))
)

analisis3.count()

128811

In [23]:
analisis4 = (
    df
    .withColumn('trip_duration', F.col('tpep_dropoff_datetime') - F.col('tpep_pickup_datetime'))
    .orderBy('trip_duration', ascending = False)
    .select('trip_duration')
    .limit(1)
    .collect()[0][0]
)

In [24]:
analisis4

datetime.timedelta(days=6, seconds=67024)

In [25]:
6*24 + 67024/3600

162.61777777777777

In [26]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-18 03:32:31--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.160.201.131, 18.160.201.126, 18.160.201.5, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.160.201.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv.1’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0.001s  

2025-03-18 03:32:32 (16.4 MB/s) - ‘taxi_zone_lookup.csv.1’ saved [12331/12331]



In [27]:
zones = (
    spark
    .read
    .csv('taxi_zone_lookup.csv', header=True)
)

In [28]:
zones.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [29]:
df.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [32]:
# Using the zone lookup data and the Yellow October 2024 data, what is the name of the LEAST frequent pickup location Zone?
zones = (
    zones
    .withColumnRenamed('LocationID', 'PULocationID')
    .select('PULocationID', 'Zone')
)

analisis5 = (
    df
    .join(zones, on='PULocationID')
    .groupby('Zone')
    .agg(F.count("*").alias('count'))
    .orderBy('count', ascending=True)
    .limit(1)
)

In [33]:
analisis5.show()

+--------------------+-----+
|                Zone|count|
+--------------------+-----+
|Governor's Island...|    1|
+--------------------+-----+



In [34]:
least_pick_up_zone = analisis5.collect()[0][0]
print(least_pick_up_zone)

Governor's Island/Ellis Island/Liberty Island
